# How RMG gets kinetics?

Han, Kehang (hkh12@mit.edu)

Nov.24, 2015

## Motivation
This post is trying to understand how RMG gets best kinetics for a reaction with a template. This understanding can be readily used for future inplementation of Evans-Polany Correction and other applications.

In [22]:
%matplotlib inline
from rmgpy.rmg.main import RMG, CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase, database
from rmgpy.data.base import ForbiddenStructureException
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy import settings
import os

In [23]:
# set-up RMG object
rmg = RMG()
rmg.reactionModel = CoreEdgeReactionModel()

# load kinetic database and forbidden structures
rmg.database = RMGDatabase()
path = os.path.join(settings['database.directory'])

# forbidden structure loading
database.loadForbiddenStructures(os.path.join(path, 'forbiddenStructures.py'))
# kinetics family Disproportionation loading
database.loadKinetics(os.path.join(path, 'kinetics'), \
                          kineticsFamilies=['Disproportionation'])

In [24]:
spcA = Species().fromSMILES("C=CC=C")
spcB = Species().fromSMILES("C=CCC")

In [25]:
newReactions = []
newReactions.extend(rmg.reactionModel.react(database, spcA, spcB))

In [47]:
rmg.reactionModel.kineticsEstimator = 'rate rules'

### Top-level of getKineticsForTemplate

In [64]:
reaction0 = newReactions[0]
template = reaction0.template
degeneracy = reaction0.degeneracy
estimator = rmg.reactionModel.kineticsEstimator
kinetics, entry = reaction0.family.getKineticsForTemplate(template, degeneracy, method=estimator)

kinetics

ArrheniusEP(A=(4.56e+14,'cm^3/(mol*s)','*|/',1.5), n=-0.7, alpha=0, E0=(0,'kcal/mol'), Tmin=(300,'K'), Tmax=(2500,'K'), comment="""Exact match found for rate rule (Cd_pri_rad;Cmethyl_Csrad)
Multiplied by reaction path degeneracy 3""")

In [63]:
reaction0 = newReactions[2]
template = reaction0.template
degeneracy = reaction0.degeneracy
estimator = rmg.reactionModel.kineticsEstimator
kinetics, entry = reaction0.family.getKineticsForTemplate(template, degeneracy, method=estimator)

kinetics

ArrheniusEP(A=(4.56e+14,'cm^3/(mol*s)','*|/',1.5), n=-0.7, alpha=0, E0=(0,'kcal/mol'), Tmin=(300,'K'), Tmax=(2500,'K'), comment="""Estimated using template (Cd_pri_rad;Cmethyl_Csrad) for rate rule (Cd_pri_rad;Cmethyl_Csrad/H/Cd)
Multiplied by reaction path degeneracy 3""")

In [68]:
reaction0 = newReactions[6]
template = reaction0.template
degeneracy = reaction0.degeneracy
estimator = rmg.reactionModel.kineticsEstimator
kinetics, entry = reaction0.family.getKineticsForTemplate(template, degeneracy, method=estimator)

kinetics

ArrheniusEP(A=(6e+11,'cm^3/(mol*s)'), n=0, alpha=0, E0=(0,'kcal/mol'), Tmin=(300,'K'), Tmax=(1500,'K'), comment="""Estimated using template (Y_rad_birad_trirad_quadrad;XH_Rrad_birad) for rate rule (C_rad/H2/Cs;C/H2/De_Csrad)
Multiplied by reaction path degeneracy 2""")

## Detailed level of getKineticsForTemplate
we got three different cases for kinetics determination:

- Exact match found (reaction0)

- Estimated using an average

- Using a more general node to estimate original template (reaction2)

- found one or more results (create one!)

In [54]:
rules = reaction0.family.rules

In [57]:
entry = rules.getRule(template)

In [60]:
entry.data.comment

''